# VGG Face - 2
* Align VGG Face -2 dataset images.

# Set the root directory.

In [0]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

# Mount google drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# Download VGG Face - 2 dataset.

### Download the test dataset.

In [0]:
!gdown --id 1ahVnKauWfcd6dj2qX1JH5YLXu-cgtGTV
!ls -al

### Extract the test dataset.

In [0]:
!rm -rf vggface2

In [0]:
!tar -xzf vggface2.tar.gz

In [0]:
!ls -al
!ls -al vggface2
!ls -al vggface2/images
!ls -al vggface2/bbox_landmark

# Align the test dataset.

In [0]:
test_image_dir = '/content/vggface2/images/test'

In [0]:
test_filename = '/content/vggface2/bbox_landmark/loose_bb_test.csv'
test_file = open(test_filename, 'rt')
test_image_bboxes = test_file.readlines()
print(len(test_image_bboxes))

### Define letter box resize function.

In [0]:
import numpy as np

def letterbox_image(input_image, target_size):
    image_height, image_width = input_image.shape[0:2]
    target_height, target_width = target_size

    scale = min(target_width/image_width, target_height/image_height)
    new_width = int(image_width*scale)
    new_height = int(image_height*scale)

    scaled_image = cv2.resize(input_image, (new_width, new_height), interpolation=cv2.INTER_CUBIC)

    output_image = np.zeros((target_height, target_width, 3), np.uint8)
    output_image.fill(128)

    start_x = (target_width - new_width) // 2
    start_y = (target_height - new_height) // 2
    output_image[start_y:(start_y + new_height), start_x:(start_x + new_width),:] = scaled_image

    return(output_image)

### Align the test dataset.

In [0]:
import cv2
from google.colab.patches import cv2_imshow

image_offset_scale = (256. - 224.) / 256. 
got_header = False
count = 0
for test_image_bbox in test_image_bboxes:  

  if(not got_header):
    got_header = True
    continue    

  count =  count + 1

  #print(test_image_bbox)
  data = test_image_bbox.split(',')
  test_image_filename = data[0].strip('"')
  test_image_filename = test_image_filename + '.jpg'
  test_image_path = os.path.join(test_image_dir, test_image_filename)

  input_image = cv2.imread(test_image_path, cv2.IMREAD_COLOR)
  image_height, image_width = input_image.shape[0:2]

  start_x = max(int(data[1]), 0)
  start_y = max(int(data[2]), 0)
  bbox_width = min(int(data[3]), image_width)
  bbox_height = min(int(data[4]), image_height)

  #print(test_image_path, start_x, start_y, bbox_width, bbox_height)  

  image_y_offset = int(image_height * image_offset_scale) / 2.
  image_x_offset = int(image_width * image_offset_scale) / 2.

  start_x_offset = int(max(start_x - image_x_offset, 0))
  start_y_offset = int(max(start_y - image_y_offset, 0))
  bbox_width_offset = int(min(bbox_width + 2 * image_x_offset, image_width))
  bbox_height_offset = int(min(bbox_height + 2 * image_y_offset, image_height))
  #print(start_x_offset, start_y_offset, bbox_width_offset, bbox_height_offset)

  #cv2_imshow(input_image)

  #cropped_image = input_image[start_y:start_y+bbox_height, start_x:start_x+bbox_width]
  #cv2_imshow(cropped_image)

  cropped_image_offset = input_image[start_y_offset:start_y_offset+bbox_height_offset, start_x_offset:start_x_offset+bbox_width_offset]
  #cv2_imshow(cropped_image_offset)  

  image_size = max(cropped_image_offset.shape[0], cropped_image_offset.shape[1])
  #print(image_size)
  output_image = letterbox_image(cropped_image_offset, (image_size, image_size))
  #cv2_imshow(output_image)

  cv2.imwrite(test_image_path, output_image)
  
print(count)

### Compress the aligned test dataset.

In [0]:
!ls -al

In [0]:
!mv vggface2 vggface2_aligned

In [0]:
!tar -czf vggface2_aligned.tar.gz vggface2_aligned

In [0]:
!cp vggface2_aligned.tar.gz '/content/drive/My Drive/datasets/.'